In [2]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

In [9]:
housing = fetch_california_housing()

In [22]:
house_X, y_house = housing.data, housing.target
m,n = house_X.shape

In [11]:
SS = StandardScaler()
X_scaled = SS.fit_transform(house_X)

In [12]:
X_scaled_B = np.c_[np.ones((m, 1)), X_scaled]

In [56]:
m

20640

<h1>Calculating gradients by hand !</h1>

In [35]:
tf.reset_default_graph()
X = tf.constant(X_scaled_B, dtype = tf.float32 ,name = 'X')
y = tf.constant(y_house.reshape(-1,1),dtype = tf.float32 ,name = 'y')
theta = tf.Variable(tf.random_uniform([n+1, 1],-1.0,1.0), name = 'theta')

y_pred = tf.matmul(X, theta, name = 'predicted_vals')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)

training_op = tf.assign(theta, theta - 0.01*gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
#         if i % 100 == 0:
#             print(mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
print(best_theta)

[[ 2.06855226]
 [ 0.83084565]
 [ 0.11975727]
 [-0.2663914 ]
 [ 0.3057718 ]
 [-0.00415385]
 [-0.03943728]
 [-0.89139402]
 [-0.86212176]]


<h1>Using optimizer and saving data!</h1>

In [46]:
tf.reset_default_graph()
X = tf.constant(X_scaled_B, dtype = tf.float32 ,name = 'X')
y = tf.constant(y_house.reshape(-1,1),dtype = tf.float32 ,name = 'y')
theta = tf.Variable(tf.random_uniform([n+1, 1],-1.0,1.0), name = 'theta')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

y_pred = tf.matmul(X, theta, name = 'predicted_vals')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
training_op = optimizer.minimize(mse)

saver = tf.train.Saver()
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        if i % 100 == 0:
            save_path = saver.save(sess, "my_model.ckpt")
#             print(mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, "my_final_model.ckpt")
print(best_theta)

[[ 2.06855774]
 [ 0.82965553]
 [ 0.11875837]
 [-0.26559547]
 [ 0.30575299]
 [-0.00450103]
 [-0.03932761]
 [-0.89980263]
 [-0.87046212]]


<h1>Loading data</h1>

In [49]:
with tf.Session() as sess:    
    data = saver.restore(sess, 'my_final_model.ckpt')

INFO:tensorflow:Restoring parameters from my_final_model.ckpt


<h1>Mini batch</h1>

In [112]:
tf.reset_default_graph()
def fetch_batch(epoch, batch_index, batch_size):
#     np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size) 
    X_batch = X_scaled_B[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch


X = tf.placeholder(dtype = tf.float32 ,name = 'X')
y = tf.placeholder(dtype = tf.float32 ,name = 'y')
theta = tf.Variable(tf.random_uniform([n+1, 1],-1.0,1.0), name = 'theta')

y_pred = tf.matmul(X, theta, name = 'predicted_vals')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
training_op = optimizer.minimize(mse)

saver = tf.train.Saver()
init = tf.global_variables_initializer()



batch_size = 100
n_epochs = 100
n_batches = int(np.ceil(m / batch_size))
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 10 == 0:
            save_path = saver.save(sess, "tmp/my_model.ckpt")
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict = {X:X_batch,y:y_batch})
    best_theta = theta.eval()
    save_path = saver.save(sess, "tmp/my_final_model.ckpt")
print(best_theta)

[[ 2.06345963]
 [ 0.81236684]
 [ 0.12666205]
 [-0.22035035]
 [ 0.31527257]
 [ 0.00393724]
 [-0.00978793]
 [-0.89503902]
 [-0.87407649]]


<h1>Loading graphs</h1>

In [119]:
tf.reset_default_graph()
saver = tf.train.import_meta_graph('tmp/my_final_model.ckpt.meta')